In [203]:
import os
from PIL import Image
import numpy as np
import time
import random as rd
import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time

In [212]:
import pandas as pd

In [214]:
pd.__version__

'1.4.2'

### **Data importation & visualization**

In [ ]:
metadata = pd.read_csv("ISIC_2020_Training_GroundTruth_v2.csv")
metadata

In [ ]:
data_dir = "data/227x227/"

data_ids = os.listdir(data_dir)
data_nb = len(os.listdir(data_dir))

In [ ]:
df = metadata[["image_name", "target"]]
X_paths = df["image_name"].values

label_0 = df.loc[df['target'] == 0]
label_1 = df.loc[df['target'] == 1]

nb_label_0 = label_0.shape[0]
nb_label_1 = label_1.shape[0]
total_labeled_data = nb_label_0 + nb_label_1

In [ ]:
print(f"Total data: {data_nb}")
print(f"Total labeled data: {total_labeled_data}")
print(f"Total unlabeled data: {data_nb - total_labeled_data}\n")

print(f"We have {nb_label_0} ({np.round(nb_label_0/total_labeled_data * 100, 2) } %) benign labeled samples")

print(f"We have {nb_label_1} ({np.round(nb_label_1/total_labeled_data * 100, 2)} %) malignant labeled samples")

In [ ]:
def reshapeImages(dim):
  if not os.path.exists(f"data/{dim}x{dim}/"):
    os.makedirs(f"data/{dim}x{dim}/")
  for idx, img_path in enumerate(os.listdir(data_dir)):
    img = Image.open(data_dir + img_path)
    new_img = img.resize((dim, dim))
    new_img.save(f"data/{dim}x{dim}/" + img_path)
    if idx % 100 == 0:
      print(idx)

In [ ]:
def getY(image_name):
  image_name = image_name.strip(".jpg")
  img = df.loc[df['image_name'] == str(image_name)]
  res = img["target"]
  return res.values[0]

In [ ]:
malignant_data = "data/malignant/"

In [ ]:
def show(id):
    return Image.open(data_dir + str(id) + ".jpg")

In [ ]:
list_img_id_label_1 = df.loc[df["target"] == 1]["image_name"].values
list_img_id_label_1

In [ ]:
getY("ISIC_0149568")

In [ ]:
# Separate malignant data
for id in list_img_id_label_1:
    os.popen(f'cp /Users/panteleiciuc/Desktop/ISEP/A3/ML_DL/skin_lesion/data/227x227/{id}.jpg /Users/panteleiciuc/Desktop/ISEP/A3/ML_DL/skin_lesion/data/malignant/')

In [ ]:
def zoom(img_id, zoom):
    img_id = str(img_id).strip(".jpg")
    img = Image.open(malignant_data + str(img_id) + ".jpg")

    alpha = zoom - 1

    width, height = img.size

    left = width * alpha
    right = width * (1-alpha)

    top = height * alpha
    bottom = height * (1-alpha)

    box = (left, top, right, bottom)

    new_img = img.crop(box).resize((width, height), Image.LANCZOS)
    new_img_name = f"{img_id}_zoom_{zoom}.jpg"

    new_img.save(malignant_data + new_img_name)

    return new_img_name

In [ ]:
def rotate(img_id):
    img_id = str(img_id).strip(".jpg")
    img = Image.open(malignant_data + str(img_id) + ".jpg")
    
    img_rot_90 = img.rotate(90)
    img_rot_180 = img.rotate(180)
    img_rot_270 = img.rotate(270)

    new_name_90 = f"{img_id}_r90.jpg"
    new_name_180 = f"{img_id}_r180.jpg"
    new_name_270 = f"{img_id}_r270.jpg"

    img_rot_90.save(malignant_data + new_name_90)
    img_rot_180.save(malignant_data + new_name_180)
    img_rot_270.save(malignant_data + new_name_270)


    # return img_rot_90, img_rot_180, img_rot_270
    return new_name_90, new_name_180, new_name_270

In [ ]:
def mirror(img_id):

    img_id = str(img_id).strip(".jpg")
    img = Image.open(malignant_data + str(img_id) + ".jpg")

    horizontal = img.transpose(Image.FLIP_LEFT_RIGHT)

    new_name = f"{img_id}_mh.jpg"
    horizontal.save(malignant_data + new_name)

    # return horizontal
    return new_name

In [ ]:
def generateNewImages(img_id):
    m_img_id = mirror(img_id)

    r90, r180, r270 = rotate(img_id)
    m_r90, m_r180, m_r270 = rotate(m_img_id)

    zoom(img_id, 1.1)
    zoom(img_id, 1.2)

    zoom(r90, 1.1)
    zoom(r90, 1.2)

    zoom(r180, 1.1)
    zoom(r180, 1.2)

    zoom(r270, 1.1)
    zoom(r270, 1.2)
    
    zoom(m_img_id, 1.1)
    zoom(m_img_id, 1.2)

    zoom(m_r90, 1.1)
    zoom(m_r90, 1.2)

    zoom(m_r180, 1.1)
    zoom(m_r180, 1.2)

    zoom(m_r270, 1.1)
    zoom(m_r270, 1.2)



In [ ]:
# Multply malignant data
# for idx, id in enumerate(list_img_id_label_1):
#     generateNewImages(id)
#     if idx % 50 == 0:
#         print(idx)

print(f"# new imgaes in the malignant directory: {len(os.listdir(malignant_data))}")

In [ ]:
# Delete original malignant images
# for id in list_img_id_label_1:
#     os.popen(f'rm /Users/panteleiciuc/Desktop/ISEP/A3/ML_DL/skin_lesion/data/malignant/{id}.jpg')

In [ ]:
new_malignant_ids = os.listdir(malignant_data)
new_malignant_ids

In [ ]:
malignant_data

In [ ]:
# put back in data dir
for idx, id in enumerate(new_malignant_ids):
    
    src = f"/Users/panteleiciuc/Desktop/ISEP/A3/ML_DL/skin_lesion/data/malignant/{id}"
    dest = "/Users/panteleiciuc/Desktop/ISEP/A3/ML_DL/skin_lesion/data/227x227/"

    os.popen(f'cp {src} {dest}') 

    if idx % 250 == 0:
        print(idx)

In [ ]:
len(os.listdir(data_dir))

In [ ]:
d = {'image_name': new_malignant_ids, 'target': [1] * len(new_malignant_ids)}


df1 = df.copy()
df2 = pd.DataFrame(data=d)

print(len(df1))
print(len(df2))
print(len(df1) + len(df2))

In [ ]:
def labelNewData(df):
    n = len(df)
    for idx,id in enumerate(new_malignant_ids):
        df.loc[n + idx] = [str(id).strip(".jpg"), 1]
    return df

In [ ]:
# Label the new images
df = labelNewData(df.copy())
df["arr"] = ""
df

In [ ]:
data_nb = len(os.listdir(data_dir))
X_ids = df["image_name"].values

label_0 = df.loc[df['target'] == 0]
label_1 = df.loc[df['target'] == 1]

nb_label_0 = label_0.shape[0]
nb_label_1 = label_1.shape[0]
total_labeled_data = nb_label_0 + nb_label_1

In [ ]:
print(f"Total data: {data_nb}")
print(f"Total labeled data: {total_labeled_data}")
print(f"Total unlabeled data: {data_nb - total_labeled_data}\n")

print(f"We have {nb_label_0} ({np.round(nb_label_0/total_labeled_data * 100, 2) } %) benign labeled samples")

print(f"We have {nb_label_1} ({np.round(nb_label_1/total_labeled_data * 100, 2)} %) malignant labeled samples")

In [ ]:
df_test = df[:2]
new_df = df_test.set_index("image_name")

In [ ]:
def JPGToArray(id):
  id = id.strip(".jpg")
  path = data_dir + id + ".jpg"
  jpg = Image.open(path)
  img_arr = np.asarray(jpg)
  return img_arr

In [ ]:
new_df

In [ ]:
def toArray():
  for id in df.index:
    df.loc[f"{id}", "arr"] = np.array([JPGToArray(id)])

In [ ]:
# toArray()

In [205]:
x = new_df[:2]

In [206]:
type(x)

pandas.core.frame.DataFrame

In [207]:
x.to_pickle('hello')

In [208]:
loaded_df = pd.read_pickle('hello')
loaded_df

,target,arr
image_name,,
ISIC_2637011,0,"[[[[192 142 141], [183 133 132], [154 104 107]..."
ISIC_0015719,0,"[[[[158 98 70], [156 96 68], [158 98 74]..."


In [ ]:
def splitData(train_percentage, validation_percentage, test_percentage):

  if train_percentage + validation_percentage + test_percentage != 1:
    return False

  img_paths = os.listdir(data_dir)[:] # limit the db

  total_data_len = len(img_paths)

  train_len = np.round(total_data_len * train_percentage).astype("int")
  validation_len = np.round(total_data_len * validation_percentage).astype("int")
  test_len = np.round(total_data_len * test_percentage).astype("int")

  rd.shuffle(img_paths)

  train_paths = img_paths[:train_len]
  validation_paths = img_paths[train_len:train_len+validation_len]
  test_paths = img_paths[train_len+validation_len:]

  return train_paths, validation_paths, test_paths

# tr_p, val_p, test_p = splitData(0.8, 0.1, 0.1)
train_paths, validation_paths, test_paths = splitData(0.8, 0.1, 0.1)

In [ ]:
def getLabels(paths):
  labels = []
  for im_path in paths:
    labels.append(getY(im_path))
  return np.resize(np.array(labels), (len(labels), 1))

# print(getLabels(val_p))

In [ ]:
def pathsToArray(paths):
  data_arr = []
  for idx,img_path in enumerate(paths):
    path = data_dir + img_path
    im_arr = np.array(Image.open(path))
    data_arr.append(im_arr)
    print(idx)
  return data_arr

In [ ]:
len(test_paths)
pathsToArray(train_paths)

In [ ]:
np.load('train_ar.npy')

In [ ]:
def bigPathToArray(paths):
  arr = []
  step = 2640
  for i in range(10):
    print("From: ", i*step)
    print("To: ", (i+1)*step)
    interm_path = paths[i*step:(i+1)*step]

    for img_path in interm_path:
      path = data_dir + img_path
      im_arr = np.array(Image.open(path))
      # arr = arr + im_arr
      arr.append(im_arr)

  return arr


# train_images = bigPathToArray(train_paths)

In [ ]:
def load_data(train_portion, val_portion, test_portion):
  train_paths, validation_paths, test_paths = splitData(train_portion, val_portion, test_portion)
  
  train_images = pathsToArray(train_paths)
  train_labels = getLabels(train_paths)

  validation_images = pathsToArray(validation_paths)
  validation_labels = getLabels(validation_paths)

  test_images = pathsToArray(test_paths)
  test_labels = getLabels(test_paths)

  return (train_images, train_labels), (validation_images, validation_labels), (test_images, test_labels)

In [ ]:
train_images = pathsToArray(train_paths)
train_labels = getLabels(train_paths)

validation_images = pathsToArray(validation_paths)
validation_labels = getLabels(validation_paths)

test_images = pathsToArray(test_paths)
test_labels = getLabels(test_paths)

#### **AlexNet Model**

In [ ]:
# (train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()
(train_images, train_labels), (validation_images, validation_labels), (test_images, test_labels) = load_data(0.8, 0.1, 0.1)

In [ ]:
print(len(train_images))
print(len(train_images[0]))
print(len(train_images[0][0]))
print(len(train_images[0][0][0]))
# print(train_labels)

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

In [ ]:
CLASS_NAMES= ["benign", "malignant"]

In [ ]:
plt.figure(figsize=(20,20))
for i, (image, label) in enumerate(train_ds.take(5)):
    ax = plt.subplot(5,5,i+1)
    plt.imshow(image)
    plt.title(CLASS_NAMES[label.numpy()[0]])
    plt.axis('off')

In [ ]:
def process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    return image, label

In [ ]:
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()

print("Training data size:", train_ds_size)
print("Test data size:", test_ds_size)
print("Validation data size:", validation_ds_size)

In [ ]:
train_ds = (train_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))

test_ds = (test_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))
                  
validation_ds = (validation_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),

    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),

    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),

    keras.layers.Flatten(),

    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),

    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),

    keras.layers.Dense(2, activation='sigmoid')
])

In [ ]:
root_logdir = os.path.join(os.curdir, "logs\\fit\\")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(train_ds,
          epochs=20,
          validation_data=validation_ds,
          validation_freq=1,
          callbacks=[tensorboard_cb])

In [ ]:
model.evaluate(test_ds)

In [ ]:
model.save_weights("./checkpoints/My_checkpoint")

In [ ]:
model.save("saved_model/my_model")